# LangChain 기본 Prompt 설명

In [1]:
# LLM Model
from langchain_openai import ChatOpenAI

# LangChain Core
from langchain_core.messages import HumanMessage, SystemMessage


model = ChatOpenAI(model="gpt-4")

messages = [
    SystemMessage(content="You are a translator, Change English to Italian"),
    HumanMessage(content="Hello!"),
]

result = model.invoke(messages)
result
# prompt 엔지니어란 이런것
# LLM에게 역할을 부여하여 원하는 대답을 끌어내는것

AIMessage(content='Ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 22, 'total_tokens': 25}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5ab95858-adae-4a13-9ef1-3b43f10260d6-0', usage_metadata={'input_tokens': 22, 'output_tokens': 3, 'total_tokens': 25})

In [2]:
# Output Parsers : 내보내는 형태 (ex: xml, json)
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [3]:
r = parser.invoke(result)
r

'Ciao!'

In [4]:
# LangChain 에 Chain 이란 이름이 붙은 이유
# 리눅스에서 쓰이는 문법 | 

chain = model | parser
chain.invoke(messages)

'Ciao!'

In [5]:
# 위에 Message 형태를 더 쉽게 -> Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a translator, Change English to {language}"),
        ("user", "{text}"),
    ]
)

chain = prompt | model | parser
chain.invoke({"language" : "korean", "text":"How are you?"})

# prompt -> prompt 내용을 그대로 model (GPT4) 에 전달
# "system" : You are a translator, Change English to korean
# "user" : How are you?

'어떻게 지내세요?'

# Retrievers 검색자 사용 방법

In [6]:
# wikipedia 정보 끌어오기
# https://python.langchain.com/v0.2/docs/integrations/retrievers/wikipedia/#question-answering-on-facts
# https://api.python.langchain.com/en/latest/retrievers/langchain_community.retrievers.wikipedia.WikipediaRetriever.html

# 위키피디아 사이트 https://www.wikipedia.org/
from langchain_community.retrievers import WikipediaRetriever

retriver = WikipediaRetriever(lang="ko", top_k_results=2)

In [7]:
docs = retriver.invoke("애플")

In [8]:
docs

[Document(page_content='애플(영어: Apple Inc.)은 미국 캘리포니아의 아이폰, 아이패드, 애플 워치, 에어팟, 아이맥, 맥북, 맥 스튜디오와 맥 프로, 홈팟, 비전 프로, 에어태그 등의 하드웨어와 그 제품들의 iOS, iPadOS, Watch OS, macOS, Vision OS, TV OS 등의 소프트웨어를 설계, 디자인하는 기업이다. 2011년부터 팀 쿡이 CEO를 맡고 있다. 또한 그 소프트웨어에 애플 앱 스토어, 뮤직, 뉴스, 아케이드, tv, 피트니스, 페이, 저장 클라우드 등의 서비스를 제공한다.\n애플의 이전 명칭은 애플 컴퓨터 주식회사(영어: Apple Computer, Inc.)였는데, 스티브 잡스가 아이팟, 아이폰을 발표하면서 간단하게 애플이라고 기업 이름을 바꾸었다. 본사는 애플 캠퍼스와 애플 파크에 두고 있으며, 미국 캘리포니아주 쿠퍼티노에 소재하고 있다. 최고경영자는 팀 쿡이다.\n2015년 2월 11일 세계 최초로 시가총액이 7,000억 달러를 넘은 기업이 되었다. 2018년 1조 달러를 넘었고 2022년 2.7조 달러가 되었다.\n\n\n== 역사 ==\n\n1976년 4월 1일: 애플 컴퓨터 주식회사(APPLE COMPUTER, INC)를 설립함.\n스티브 워즈니악과 스티브 잡스, 론 웨인이 잡스의 부모님의 차고에서 컴퓨터 조립 키트인 \'애플 I\'을 만들며 애플이 시작되었다.\n1977년 4월 17일: 애플 II가 출시되었다.\n1979년 1월 3일: 애플 컴퓨터 주식회사가 주식 상장을 하였다.\n1983년 1월 19일: 애플 리사가 출시되었다.\n1984년 1월 24일: 매킨토시가 처음 공개되었다. \'1984년 광고\'가 방영되었다.\n1985년: 스티브 잡스가 존 스컬리에 의해 애플에서 해고되었다.\n1993년: 세계 최초의 PDA이자 태블릿 플랫폼인 애플 뉴턴을 출시했다.\n1997년: 스티브 잡스가 다시 애플로 돌아와 다시 애플의 CEO가 되었다.\n1998년 8월 15일: 아이맥이 공개되었다

In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are an explanatory expert, Do not explain text that is not included.
            Just write answers to user questions.
            Explain it in Korean
            -----
            {context}
            -----
            user question: {question} 
            """,
        ),
    ]
)

In [10]:
# 문서가 나누어져 잇으니 벡터로 검색하기
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

db = Chroma.from_documents(docs, OpenAIEmbeddings())

In [14]:
query = "애플은 어디에 있는 회사인가?"
doc_list = db.similarity_search(query)
answer = doc_list[0].page_content

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


In [18]:
chain = prompt | model | parser
# chain.invoke({"context": answer, "question":})

'애플(Apple Inc.)은 미국 캘리포니아에 본사를 둔 기업으로, 아이폰, 아이패드, 애플 워치, 에어팟, 아이맥, 맥북, 맥 스튜디오와 맥 프로, 홈팟, 비전 프로, 에어태그 등 다양한 하드웨어를 설계하고 제작합니다. 또한 이런 제품들을 동작시키는 iOS, iPadOS, Watch OS, macOS, Vision OS, TV OS 등의 소프트웨어도 개발하고 있습니다. 2011년부터 팀 쿡이 CEO를 맡고 있으며, 소프트웨어에는 애플 앱 스토어, 뮤직, 뉴스, 아케이드, tv, 피트니스, 페이, 저장 클라우드 등의 서비스를 제공하고 있습니다.\n\n애플의 이전 명칭은 애플 컴퓨터 주식회사(Apple Computer, Inc.)였습니다. 스티브 잡스가 아이팟, 아이폰을 발표하면서 간단하게 애플로 기업 이름을 바꿨습니다. 본사는 애플 캠퍼스와 애플 파크에 위치해 있으며, 미국 캘리포니아주 쿠퍼티노에 소재하고 있습니다. 최고경영자는 팀 쿡입니다.\n\n애플은 1976년 4월 1일에 설립되었고, 스티브 워즈니악과 스티브 잡스, 론 웨인이 잡스의 부모님의 차고에서 컴퓨터 조립 키트인 \'애플 I\'을 만들었습니다. 그 이후로 다양한 제품을 출시하며 세계 최초로 시가총액이 7,000억 달러를 넘은 기업이 되었습니다. 2018년에는 1조 달러를 넘었고, 2022년에는 2.7조 달러가 되었습니다.\n\n애플은 맥, 아이폰, 애플 워치, 아이패드 등 다양한 제품군을 생산하고 있습니다. 맥은 사용의 편의성과 독특한 알루미늄 미니멀리즘 디자인으로 유명하며, 아이폰은 iOS 운영체제를 실행하는 스마트폰 라인으로, 그 멀티 터치 스크린은 휴대폰 산업의 "혁명적"이고 "게임 체인저"로 묘사되었습니다. 아이패드는 아이패드 OS를 구동하는 태블릿 라인으로, 주로 멀티미디어 소비, 예술 창작, 문서 작업, 화상 회의, 게임 등을 목적으로 판매되고 있습니다.'

# 텍스트로 질문하고 그 중 단어를 검색 후 답변하기

In [19]:
keyword_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Text blocks are provided, and the task is to extract one of the most important keywords from the text blocks.
            Extract the language you used
            -----
            {question}
            """,
        ),
    ]
)

In [22]:
user_question = "테슬라 기업은 언제 설립하였나?"

keyword_chain = keyword_prompt | model | parser
result = keyword_chain.invoke({"question": user_question})
print(result)

테슬라 기업


In [25]:
docs = retriver.invoke(result)
db = Chroma.from_documents(docs, OpenAIEmbeddings())

doc_list = db.similarity_search(user_question)
answer = doc_list[0].page_content


chain = prompt | model | parser
chain.invoke({"context": answer, "question": user_question})

/Users/baekseung-gyu/Documents/LectureProjects/NewsGPT/venv/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/baekseung-gyu/Documents/LectureProjects/NewsGPT/venv/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


'테슬라는 2003년에 마틴 에버하드(CEO)와 마크 타페닝(CFO)이 창업한 회사입니다.'